# Problem 2: Getting familiar with TensorFlow

*TensorFlow* is one of the most popular deep learning framework developed by Google. If you are new to TensorFlow, please read and play with the sample in [Getting started with TensorFlow](https://www.tensorflow.org/get_started/get_started) to get started.

* <b>Learning Objective:</b> In Problem 1, you implemented a fully connected network from scratch on your own. Very tedious to do it all by yourself, right? Well, we actually feel the same thing, that's why we are using tools instead of doing everything from scratch. For this part of the assignment, we will familiarize you with a widely-used deep learning framework developed by Google, TensorFlow and walk you through convolutional neural networks and show how to train them.
* <b>Provided Codes:</b> We provide the Template class for a simple CNN model as BaseModel, predefined skeletons for conv2d() and max_pool(), as well as the dataset preprocessing parts.
* <b>TODOs:</b> You are asked to implement the BaseModel following the detailed instructions and design your own model in YourModel to achieve a reasonably good performance for classification task on CIFAR-10 dataset.

## Import libraries

In [2]:
# Import required libraries
# Add whatever you want
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from lib.datasets import CIFAR10_tf

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# We recommend to use tensorflow==1.14.0
print("TensorFlow Version {}".format(tf.__version__))

TensorFlow Version 1.14.0


## Load datasets
Download [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) and load the dataset. In this assignment, we will use the standard 50,000 images for training and 10,000 images for test.

In [4]:
# Configuration
num_training = 49000
num_validation = 50000 - num_training
num_test = 10000

data = CIFAR10_tf(num_training=num_training,
                  num_validation=num_validation,
                  num_test=num_test)

# Load cifar-10 data
X_train, Y_train = data['data_train'], data['labels_train']
X_val, Y_val = data['data_val'], data['labels_val']
X_test, Y_test = data['data_test'], data['labels_test']

# Check the shape of the dataset
assert X_train.shape == (num_training, 32, 32, 3)
assert Y_train.shape == (num_training, )
assert X_val.shape == (num_validation, 32, 32, 3)
assert Y_val.shape == (num_validation, )
assert X_test.shape == (num_test, 32, 32, 3)
assert Y_test.shape == (10000, )

In [7]:
from collections import Counter

Y_train_count = Counter(Y_train)
Y_val_count = Counter(Y_val)
Y_test_count = Counter(Y_test)

print(f'Y_train_count: {Y_train_count}')
print(f'Y_val_count: {Y_val_count}')
print(f'Y_test_count: {Y_test_count}')

Y_train_count: Counter({4: 4922, 2: 4921, 0: 4913, 5: 4902, 6: 4898, 9: 4895, 7: 4893, 3: 4888, 8: 4887, 1: 4881})
Y_val_count: Counter({1: 119, 8: 113, 3: 112, 7: 107, 9: 105, 6: 102, 5: 98, 0: 87, 2: 79, 4: 78})
Y_test_count: Counter({3: 1000, 8: 1000, 0: 1000, 6: 1000, 1: 1000, 9: 1000, 5: 1000, 7: 1000, 4: 1000, 2: 1000})


## Question 2-1 [10pt]

Using the code provided, implement a neural network architecture with an optimization routine according to the specification provided below.

**Model:**
- Input image with the size 32x32x3
- 7x7 convolutional layer with 32 filters, stride of 1, and padding 'SAME'
- ReLU activation layer
- 3x3 max pooling layer with a stride of 2
- 5x5 convolutional layer with 64 filters, stride of 1, and padding 'SAME'
- ReLU activation layer
- 3x3 max pooling layer with a stride of 2
- Flatten layer (8x8x64 -> 4096)
- Fully-connected layer with 384 output units (4096 -> 384)
- ReLU activation layer
- Fully-connected layer with 10 output units (384 -> 10)
- Output logits (10)

**Optimizer:**
- Adam optimizer

**Learning rate:**
- Set start learning rate as 5e-4 and apply exponential decay every 500 steps with a base of 0.96
- Use 'tf.train.exponential_decay' and 'tf.train.AdamOptimizer'

**Loss:**
- Softmax cross entropy loss
- Use 'tf.nn.softmax_cross_entropy_with_logits_v2'


Your model **should** achieve about 55% accuracy on test set in 5 epochs using provided evaluation code.

You can modify the template code as you want.

## Define your layers

In [61]:
# Define max pooling and conv layers

def conv2d(input, kernel_size, stride, num_filter):
    stride_shape = [1, stride, stride, 1]
    filter_shape = [kernel_size, kernel_size, input.get_shape()[3], num_filter]
    W = tf.get_variable('w', filter_shape, tf.float32, tf.random_normal_initializer(0.0, 0.02))
    b = tf.get_variable('b', [1, 1, 1, num_filter], initializer=tf.constant_initializer(0.0))
    return tf.nn.conv2d(input, W, stride_shape, padding='SAME') + b

def max_pool(input, kernel_size, stride):
    ksize = [1, kernel_size, kernel_size, 1]
    strides = [1, stride, stride, 1]
    return tf.nn.max_pool(input, ksize=ksize, strides=strides, padding='SAME')

#############################################################################
# TODO: Complete the following functions                                    #
#############################################################################
def flatten(input):
    """
        - input: input tensors
    """
    
    return tf.layers.flatten(input)

def fc(input, num_output):
    """
        - input: input tensors
        - num_output: int, the output dimension
    """
    regularizer = tf.contrib.layers.l2_regularizer(scale = 0.0001)
    return tf.contrib.layers.fully_connected(input, num_output, activation_fn = None, weights_regularizer = regularizer)

def conv2d_regularized(input, kernel_size, stride, num_filter):
    regularizer = tf.contrib.layers.l2_regularizer(scale = 0.0001)
    return tf.layers.conv2d(input, num_filter, kernel_size, stride, kernel_initializer = tf.random_normal_initializer(0.0, 0.02), kernel_regularizer = regularizer, padding='same')
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################

## Sample convolutional neural network

In [62]:
class BaseModel(object):
    def __init__(self):
        self.num_epoch = 5
        self.batch_size = 128
        self.log_step = 100
        self._build_model()

    def _model(self):
        print('-' * 5 + '  Sample model  ' + '-' * 5)

        print('intput layer: ' + str(self.X.get_shape()))

        with tf.variable_scope('conv1'):
            self.conv1 = conv2d(self.X, 7, 1, 32)
            self.relu1 = tf.nn.relu(self.conv1)
            self.pool1 = max_pool(self.relu1, 3, 2)            
            print('conv1 layer: ' + str(self.pool1.get_shape()))

        with tf.variable_scope('conv2'):
            #############################################################################
            # TODO: Complete the following functions                                    #
            #############################################################################
            self.conv2 = conv2d(self.pool1, 5, 1, 64)
            self.relu2 = tf.nn.relu(self.conv2)
            self.pool2 = max_pool(self.relu2, 3, 2)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            print('conv2 layer: ' + str(self.pool2.get_shape()))

        #############################################################################
        # TODO: Flatten the output tensor from conv2 layer                          #
        #############################################################################
        self.flat = flatten(self.pool2)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################      
        print('flat layer: ' + str(self.flat.get_shape()))

        with tf.variable_scope('fc3'):
            #############################################################################
            # TODO: Complete the following functions                                    #
            #############################################################################
            self.fc3 = fc(self.flat, 384)
            self.relu3 = tf.nn.relu(self.fc3)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            print('fc3 layer: ' + str(self.relu3.get_shape()))
            
        with tf.variable_scope('fc4'):
            #############################################################################
            # TODO: Complete the following functions                                    #
            #############################################################################
            self.fc4 = fc(self.relu3, 10)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            print('fc4 layer: ' + str(self.fc4.get_shape()))
            
        # Return the last layer
        return self.fc4

    def _input_ops(self):
        # Placeholders
        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.Y = tf.placeholder(tf.int64, [None])

        #############################################################################
        # TODO: You can add any placeholders                                        #
        #############################################################################
        
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def _build_optimizer(self):
        #############################################################################
        # TODO: Adam optimizer 'self.train_op' that minimizes 'self.loss_op'        #
        #############################################################################
        self.step = tf.get_variable(name = 'step', dtype = tf.int32, initializer = 0)
        self.rate = tf.train.exponential_decay(5e-4, self.step, 500, 0.96)
        self.train_op = tf.train.AdamOptimizer(self.rate).minimize(self.loss_op, global_step=self.step)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        
    def _loss(self, labels, logits):
        #############################################################################
        # TODO: Softmax cross entropy loss 'self.loss_op'                           #
        #############################################################################
        self.loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels, logits, name='loss'))
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def _build_model(self):
        # Define input variables
        self._input_ops()

        # Convert Y to one-hot vector
        labels = tf.one_hot(self.Y, 10)

        # Build a model and get logits
        logits = self._model()

        # Compute loss
        self._loss(labels, logits)
        
        # Build optimizer
        self._build_optimizer()

        # Compute accuracy
        predict = tf.argmax(logits, 1)
        correct = tf.equal(predict, self.Y)
        self.accuracy_op = tf.reduce_mean(tf.cast(correct, tf.float32))
        
    def train(self, sess, X_train, Y_train, X_val, Y_val):
        sess.run(tf.global_variables_initializer())

        step = 0
        losses = []
        accuracies = []
        print('-' * 5 + '  Start training  ' + '-' * 5)
        for epoch in range(self.num_epoch):
            print('train for epoch %d' % epoch)
            for i in range(num_training // self.batch_size):
                X_ = X_train[i * self.batch_size:(i + 1) * self.batch_size][:]
                Y_ = Y_train[i * self.batch_size:(i + 1) * self.batch_size]

                #############################################################################
                # TODO: You can change feed data as you want                                #
                #############################################################################
                feed_dict = {self.X: X_, self.Y: Y_}
                #############################################################################
                #                             END OF YOUR CODE                              #
                #############################################################################
                fetches = [self.train_op, self.loss_op, self.accuracy_op]

                _, loss, accuracy = sess.run(fetches, feed_dict=feed_dict)
                losses.append(loss)
                accuracies.append(accuracy)

                if step % self.log_step == 0:
                    print('iteration (%d): loss = %.3f, accuracy = %.3f' %
                        (step, loss, accuracy))
                step += 1

            # Print validation results
            print('validation for epoch %d' % epoch)
            val_accuracy = self.evaluate(sess, X_val, Y_val)
            print('-  epoch %d: validation accuracy = %.3f' % (epoch, val_accuracy))
            
        #############################################################################
        # TODO: Plot training curve                                                 #
        #############################################################################
        # Graph 1. X: iteration (training step), Y: training loss
        plt.subplot(2,1,1)
        plt.title('Training loss vs Iterations')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.plot(losses, '-o')
        # Graph 2. X: iteration (training step), Y: training accuracy
        
        plt.subplot(2,1,2)
        plt.title('Training accuracy vs Iterations')
        plt.xlabel('Iterations')
        plt.ylabel('Accuracy')
        plt.plot(accuracies, '-o')
        
        plt.gcf().set_size_inches(15, 15)
        plt.show()
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def evaluate(self, sess, X_eval, Y_eval):
        eval_accuracy = 0.0
        eval_iter = 0
        for i in range(X_eval.shape[0] // self.batch_size):
            X_ = X_eval[i * self.batch_size:(i + 1) * self.batch_size][:]
            Y_ = Y_eval[i * self.batch_size:(i + 1) * self.batch_size]
            
            #############################################################################
            # TODO: You can change feed data as you want                                #
            #############################################################################
            feed_dict = {self.X: X_, self.Y: Y_}
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            accuracy = sess.run(self.accuracy_op, feed_dict=feed_dict)
            eval_accuracy += accuracy
            eval_iter += 1
        return eval_accuracy / eval_iter

In [ ]:
# Clear old computation graphs
tf.reset_default_graph()

# Train our sample model
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/cpu:0'):
        model = BaseModel()
        model.train(sess, X_train, Y_train, X_val, Y_val)
        accuracy = model.evaluate(sess, X_test, Y_test)
        print('***** test accuracy: %.3f' % accuracy)
        saver = tf.train.Saver()
        model_path = saver.save(sess, "lib/tf_models/problem2/csci-599_sample.ckpt")
        print("Model saved in %s" % model_path)

## Question 2-2 [15pt]

Implement your own model. 

You can modify the template code as you want and you can use GPU for fast training. For GPU usage, simply change the following line of the training block:  
from `with tf.device('/cpu:0')` to `with tf.device('/GPU:0')` and you can set your desired device number.

These are the techniques that you can try:
- Data preprocessing
- Data augmentation
- Batch normalization
- Dropout
- More convolutional layers
- More training epochs
- Learning rate decay
- Any other models and techniqes

The rubrics for this question is:
* 15 points when test accuracy >= 75%
* 10 points when test accuracy >= 70%
* 5 points when test accuracy >= 60%

In [66]:
import time
class YourModel(BaseModel):
    def __init__(self):
        super(YourModel, self).__init__()
        self.num_epoch = 50
        self.principal_components = None
        self.zca_epsilon = 1e-6
    
    def _input_ops(self):
        # Placeholders
        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.Y = tf.placeholder(tf.int64, [None])
        self.training = tf.placeholder_with_default(False, shape=(), name='training')
        
    def _loss(self, labels, logits):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels, logits, name='vanilla_loss'))
        l2_regularizer_loss = tf.losses.get_regularization_loss()
        self.reg_loss_op = l2_regularizer_loss
        self.loss_op = loss + l2_regularizer_loss

    def _model(self):
        print('-' * 5 + '  Your model  ' + '-' * 5)

        #############################################################################
        # TODO: Implement you own model here                                        #
        #############################################################################
        self.keep_prob = 0.5
        print('intput layer: ' + str(self.X.get_shape()))
        
        with tf.variable_scope('conv1'):
            self.conv1 = conv2d_regularized(self.X, 7, 1, 32)
            self.bn1 = tf.layers.batch_normalization(self.conv1, training = self.training)
            self.elu1 = tf.nn.elu(self.bn1)
            self.do1 = tf.nn.dropout(self.elu1, keep_prob = self.keep_prob)
            self.pool1 = max_pool(self.do1, 3, 2)            
            print('conv1 layer: ' + str(self.pool1.get_shape()))

        with tf.variable_scope('conv2'):
            #############################################################################
            # TODO: Complete the following functions                                    #
            #############################################################################
            self.conv2 = conv2d_regularized(self.pool1, 5, 1, 64)
            self.bn2 = tf.layers.batch_normalization(self.conv2, training = self.training)
            self.elu2 = tf.nn.relu(self.bn2)
            self.do2 = tf.nn.dropout(self.elu2, keep_prob = self.keep_prob)
            self.pool2 = max_pool(self.do2, 3, 2)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            print('conv2 layer: ' + str(self.pool2.get_shape()))

        #############################################################################
        # TODO: Flatten the output tensor from conv2 layer                          #
        #############################################################################
        self.flat = flatten(self.pool2)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################      
        print('flat layer: ' + str(self.flat.get_shape()))

        with tf.variable_scope('fc3'):
            #############################################################################
            # TODO: Complete the following functions                                    #
            #############################################################################
            self.fc3 = fc(self.flat, 384)
            self.bn3 = tf.layers.batch_normalization(self.fc3, training = self.training)
            self.elu3 = tf.nn.elu(self.bn3)
            self.do3 = tf.nn.dropout(self.elu3, keep_prob = self.keep_prob)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            print('fc3 layer: ' + str(self.elu3.get_shape()))
            
        with tf.variable_scope('fc4'):
            #############################################################################
            # TODO: Complete the following functions                                    #
            #############################################################################
            self.fc4 = fc(self.do3, 10)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            print('fc4 layer: ' + str(self.fc4.get_shape()))
            
        # Return the last layer
        return self.fc4
        
        
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        
    def fit_zca(self, x):
        flat_x = np.reshape(
                x, (x.shape[0], x.shape[1] * x.shape[2] * x.shape[3]))
        sigma = np.dot(flat_x.T, flat_x) / flat_x.shape[0]
        u, s, _ = np.linalg.svd(sigma)
        s_inv = 1. / np.sqrt(s[np.newaxis] + self.zca_epsilon)
        self.principal_components = (u * s_inv).dot(u.T)
    
    def compute_projection(self, x):
        flatx = np.reshape(x, (-1, np.prod(x.shape[-3:])))
        whitex = np.dot(flatx, self.principal_components)
        x_trans = np.reshape(whitex, x.shape)
        return x_trans
    
    def train(self, sess, X_train, Y_train, X_val, Y_val):
        sess.run(tf.global_variables_initializer())
        self.fit_zca(X_train)
        X_train_transformed = self.compute_projection(X_train)
        step = 0
        losses = []
        accuracies = []
        reg_losses = []
        last_time = time.time()
        train_start = last_time
        print('-' * 5 + '  Start training  ' + '-' * 5)
        for epoch in range(self.num_epoch):
            print('train for epoch %d' % epoch)
            for i in range(num_training // self.batch_size):
                X_ = X_train_transformed[i * self.batch_size:(i + 1) * self.batch_size][:]
                Y_ = Y_train[i * self.batch_size:(i + 1) * self.batch_size]
                #############################################################################
                # TODO: You can change feed data as you want                                #
                #############################################################################
                feed_dict = {self.X: X_, self.Y: Y_}
                #############################################################################
                #                             END OF YOUR CODE                              #
                #############################################################################
                fetches = [self.train_op, self.loss_op, self.accuracy_op, self.reg_loss_op]

                _, loss, accuracy, reg_loss = sess.run(fetches, feed_dict=feed_dict)
                losses.append(loss)
                accuracies.append(accuracy)
                reg_losses.append(reg_loss)
                if step % self.log_step == 0:
                    now = time.time()
                    print('iteration (%d): loss = %.3f, accuracy = %.3f, reg_loss: %.3f' %
                        (step, loss, accuracy, reg_loss))
                    print(f'Average iteration time is: {(now - last_time)/self.log_step:.3f}')
                    print(f'Total time elapsed is {now - train_start:.3f}')
                    
                step += 1

            # Print validation results
            print('validation for epoch %d' % epoch)
            start = time.time()
            val_accuracy = self.evaluate(sess, X_val, Y_val)
            end = time.time()
            print('-  epoch %d: validation accuracy = %.3f' % (epoch, val_accuracy))
            print(f'Time taken for evaluation is {end-start}')
        
        train_end_time = time.time()
        print(f'Total training time is {train_end_time - train_start:.3f}')
        
        #############################################################################
        # TODO: Plot training curve                                                 #
        #############################################################################
        # Graph 1. X: iteration (training step), Y: training loss
        plt.subplot(2,1,1)
        plt.title('Training loss vs Iterations')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.plot(losses, '-o')
        # Graph 2. X: iteration (training step), Y: training accuracy
        
        plt.subplot(2,1,2)
        plt.title('Training accuracy vs Iterations')
        plt.xlabel('Iterations')
        plt.ylabel('Accuracy')
        plt.plot(accuracies, '-o')
        
        plt.gcf().set_size_inches(15, 15)
        plt.show()
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
    
    def evaluate(self, sess, X_eval, Y_eval):
        eval_accuracy = 0.0
        eval_iter = 0
        for i in range(X_eval.shape[0] // self.batch_size):
            X_ = X_eval[i * self.batch_size:(i + 1) * self.batch_size][:]
            Y_ = Y_eval[i * self.batch_size:(i + 1) * self.batch_size]
            X_ = self.compute_projection(X_)
            #############################################################################
            # TODO: You can change feed data as you want                                #
            #############################################################################
            feed_dict = {self.X: X_, self.Y: Y_}
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            accuracy = sess.run(self.accuracy_op, feed_dict=feed_dict)
            eval_accuracy += accuracy
            eval_iter += 1
        return eval_accuracy / eval_iter

In [ ]:
# Clear old computation graphs
tf.reset_default_graph()

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/cpu:0'):
        model = YourModel()
        model.train(sess, X_train, Y_train, X_val, Y_val)
        accuracy = model.evaluate(sess, X_test, Y_test)
        print('***** test accuracy: %.3f' % accuracy)
        # Save your model
        saver = tf.train.Saver()
        model_path = saver.save(sess, "lib/tf_models/problem2/csci-599_mine.ckpt")
        print("Model saved in %s" % model_path)

-----  Your model  -----
intput layer: (?, 32, 32, 3)
conv1 layer: (?, 16, 16, 32)
conv2 layer: (?, 8, 8, 64)
flat layer: (?, 4096)
fc3 layer: (?, 384)
fc4 layer: (?, 10)
-----  Start training  -----
train for epoch 0
iteration (0): loss = 2.727, accuracy = 0.117, reg_loss: 0.037
Average iteration time is: 0.004
Total time elapsed is 0.426
iteration (100): loss = 2.041, accuracy = 0.195, reg_loss: 0.036
Average iteration time is: 0.250
Total time elapsed is 25.019
iteration (200): loss = 1.924, accuracy = 0.344, reg_loss: 0.035
Average iteration time is: 0.474
Total time elapsed is 47.373
iteration (300): loss = 1.615, accuracy = 0.438, reg_loss: 0.034
Average iteration time is: 0.713
Total time elapsed is 71.323
validation for epoch 0
-  epoch 0: validation accuracy = 0.396
Time taken for evaluation is 1.244807243347168
train for epoch 1
iteration (400): loss = 1.620, accuracy = 0.398, reg_loss: 0.033
Average iteration time is: 0.964
Total time elapsed is 96.398
iteration (500): loss 

validation for epoch 12
-  epoch 12: validation accuracy = 0.740
Time taken for evaluation is 1.3539912700653076
train for epoch 13
iteration (5000): loss = 0.489, accuracy = 0.852, reg_loss: 0.045
Average iteration time is: 13.172
Total time elapsed is 1317.199
iteration (5100): loss = 0.605, accuracy = 0.789, reg_loss: 0.045
Average iteration time is: 13.460
Total time elapsed is 1346.041
iteration (5200): loss = 0.502, accuracy = 0.828, reg_loss: 0.046
Average iteration time is: 13.749
Total time elapsed is 1374.862
iteration (5300): loss = 0.657, accuracy = 0.766, reg_loss: 0.047
Average iteration time is: 14.042
Total time elapsed is 1404.176
validation for epoch 13
-  epoch 13: validation accuracy = 0.737
Time taken for evaluation is 1.31243896484375
train for epoch 14
iteration (5400): loss = 0.660, accuracy = 0.758, reg_loss: 0.048
Average iteration time is: 14.348
Total time elapsed is 1434.824
iteration (5500): loss = 0.677, accuracy = 0.789, reg_loss: 0.048
Average iteration

validation for epoch 25
-  epoch 25: validation accuracy = 0.737
Time taken for evaluation is 1.1729979515075684
train for epoch 26
iteration (10000): loss = 0.308, accuracy = 0.914, reg_loss: 0.068
Average iteration time is: 26.779
Total time elapsed is 2677.930
iteration (10100): loss = 0.377, accuracy = 0.898, reg_loss: 0.068
Average iteration time is: 27.062
Total time elapsed is 2706.205
iteration (10200): loss = 0.302, accuracy = 0.906, reg_loss: 0.068
Average iteration time is: 27.339
Total time elapsed is 2733.930
iteration (10300): loss = 0.457, accuracy = 0.844, reg_loss: 0.068
Average iteration time is: 27.627
Total time elapsed is 2762.680
validation for epoch 26
-  epoch 26: validation accuracy = 0.766
Time taken for evaluation is 1.2530317306518555
train for epoch 27
iteration (10400): loss = 0.421, accuracy = 0.891, reg_loss: 0.069
Average iteration time is: 27.991
Total time elapsed is 2799.150
iteration (10500): loss = 0.346, accuracy = 0.906, reg_loss: 0.069
Average i

Model specifications
1. With BN: test acc: 0.61
2. with BN and dropout: 0.644

In [ ]:
tf.reset_default_graph()

# Load your model
model = YourModel()
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, "lib/tf_models/problem2/csci-599_mine.ckpt")